# Robert Chatterton

#### DS 4400 Machine Learning and Data Mining I Final Project

<hr />

## Authorship Attribution on Classical Works and More

`Outline:`

    - Clean and pull data
        - Read in texts from NLTK
        - Choose specific ones (5 authors?)

    - Prepare data
        - N-grams
        - Bag of Words
        - word2Vec? if I can
        - Get style
            - sentence length
            - word choice
            - sentiment
            

    - Models:
        - Logistic Regression
        - Random Forest
        - Naive Bayes
        - NN (LSTM or Feed forward?)

In [19]:
# cleaning imports
import numpy as np
import nltk
import random
import pandas as pd

RANDOM_SEED = 3
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Data Cleaning

In [3]:
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /home/robbie/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /home/robbie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
minimum_sentence_length = 5

def create_sentence_list(work):
    raw = nltk.corpus.gutenberg.raw(work)
    
    # fix up text
    text = raw.replace('."', '".').replace('?"', '"?').replace('!"', '"!')
    text = text.replace('--', ' ').replace('. . .', '').replace('_', '')
    text = text.replace('\n', ' ').replace('  ', ' ').replace('  ', ' ')

    # tokenize into sentences
    sents = nltk.tokenize.sent_tokenize(text)

    # remove short sentences
    sents = [sent for sent in sents if len(sent) > minimum_sentence_length]
    print(f'{work} has {len(sents)} sentences')
    return sents

# authors and works to investigate
austen = create_sentence_list('austen-persuasion.txt') + create_sentence_list('austen-sense.txt')
chesterson = create_sentence_list('chesterton-ball.txt') + create_sentence_list('chesterton-brown.txt') + create_sentence_list('chesterton-thursday.txt')
shakespeare = create_sentence_list('shakespeare-caesar.txt') + create_sentence_list('shakespeare-hamlet.txt') + create_sentence_list('shakespeare-macbeth.txt')

works = {
    'austen': austen, 
    'chesterson': chesterson, 
    'shakespeare': shakespeare
}

for a, w in works.items():
    print(f'{a} has {len(w)} sentences total')

austen-persuasion.txt has 3599 sentences
austen-sense.txt has 5134 sentences
chesterton-ball.txt has 4591 sentences
chesterton-brown.txt has 3706 sentences
chesterton-thursday.txt has 3554 sentences
shakespeare-caesar.txt has 1423 sentences
shakespeare-hamlet.txt has 2048 sentences
shakespeare-macbeth.txt has 1284 sentences
austen has 8733 sentences total
chesterson has 11851 sentences total
shakespeare has 4755 sentences total


Choose random sentences for training

In [18]:
# slightly lower than least amount of sentences of any author
to_choose = 4500
sentences = []
sentence_labels = []

for author, sents in works.items():
    chosen = np.random.choice(sents, to_choose, replace=False)
    sentences += list(chosen)
    sentence_labels += [author] * to_choose

# shuffle list randomly
Xy = list(zip(sentences, sentence_labels))
random.shuffle(Xy)
X, y = zip(*Xy)

df = pd.DataFrame()
df['sentence'] = X
df['author'] = y
df

,sentence,author
0,Mr. Cumberland Vane laughed outright.,chesterson
1,"""Something very melancholy must be the matter,...",austen
2,"You will tell Sir Walter what we have done, an...",austen
3,"My Crowne, mine owne Ambition, and my Queene: ...",shakespeare
4,Is it not like the King?,shakespeare
...,...,...
13495,He had a violent run of a hundred yards to rea...,chesterson
13496,"""Marianne's note, by assuring me that I was st...",austen
13497,He put this carefully in his upper waistcoat p...,chesterson
13498,If he steale ought the whil'st this Play is Pl...,shakespeare


# Prepare Data